In [1]:
using Pkg
Pkg.add("StatsBase")
Pkg.add("ClassImbalance")
Pkg.add("ROCAnalysis")
using Pkg
using DataFrames
using CSV
using Plots
using Lathe
using GLM
using StatsBase
using MLBase
using ClassImbalance
using ROCAnalysis

   Updating registry at `C:\Users\sdeo7\.julia\registries\General`
  Resolving package versions...
Updating `C:\Users\sdeo7\.julia\environments\v1.5\Project.toml`
  [2913bbd2] + StatsBase v0.33.5
No Changes to `C:\Users\sdeo7\.julia\environments\v1.5\Manifest.toml`
  Resolving package versions...
  Installed Lathe ────────────────── v0.0.9
  Installed KernelDensity ────────── v0.5.1
  Installed UnPack ───────────────── v1.0.2
  Installed ArrayInterface ───────── v2.14.17
  Installed ForwardDiff ──────────── v0.10.18
  Installed ClassImbalance ───────── v0.8.7
  Installed StatsBase ────────────── v0.32.2
  Installed DataFrames ───────────── v0.20.2
  Installed NLSolversBase ────────── v7.5.0
  Installed SpecialFunctions ─────── v0.10.3
  Installed CommonSubexpressions ─── v0.3.0
  Installed Tables ───────────────── v1.4.2
  Installed DataStructures ───────── v0.17.20
  Installed DiffEqDiffTools ──────── v1.7.0
  Installed CategoricalArrays ────── v0.7.7
  Installed FillArrays ──────────

In [2]:
# Enable Printing of 1000 columns
ENV["COLUMNS"] = 1000

1000

In [3]:
df = DataFrame(CSV.File("Churn_Modelling.csv"))
first(df,5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
,Int64,Int64,String,Int64,String,String,Int64,Int64,Float64,Int64,Int64,Int64,Float64,Int64
1,1,15634602,Hargrave,619,France,Female,42,2,0.0,1,1,1,1.01349e5,1
2,2,15647311,Hill,608,Spain,Female,41,1,83807.9,1,0,1,1.12543e5,0
3,3,15619304,Onio,502,France,Female,42,8,1.59661e5,3,1,0,1.13932e5,1
4,4,15701354,Boni,699,France,Female,39,1,0.0,2,0,0,93826.6,0
5,5,15737888,Mitchell,850,Spain,Female,43,2,1.25511e5,1,1,1,79084.1,0


In [4]:
println(size(df))

describe(df)

(10000, 14)


,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,RowNumber,5000.5,1,5000.5,10000,,,Int64
2,CustomerId,1.56909e7,15565701,1.56907e7,15815690,,,Int64
3,Surname,,Abazu,,Zuyeva,2932,,String
4,CreditScore,650.529,350,652.0,850,,,Int64
5,Geography,,France,,Spain,3,,String
6,Gender,,Female,,Male,2,,String
7,Age,38.9218,18,37.0,92,,,Int64
8,Tenure,5.0128,0,5.0,10,,,Int64
9,Balance,76485.9,0.0,97198.5,2.50898e5,,,Float64


In [5]:
names(df) #Names of The columns

14-element Array{Symbol,1}:
 :RowNumber
 :CustomerId
 :Surname
 :CreditScore
 :Geography
 :Gender
 :Age
 :Tenure
 :Balance
 :NumOfProducts
 :HasCrCard
 :IsActiveMember
 :EstimatedSalary
 :Exited

In [7]:
# Count the Classes
countmap(df.Exited)
# The Countmap function counts the number of unique entries in the dataframe's column

Dict{Int64,Int64} with 2 entries:
  0 => 7963
  1 => 2037

In [8]:
Lathe.preprocess.OneHotEncode(df,:Geography)
Lathe.preprocess.OneHotEncode(df,:Gender)
select!(df, Not([:RowNumber, :CustomerId, :Surname, :Geography, :Gender, :Male]))
first(df,5)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,France,Spain,Germany,Female
,Int64,Int64,Int64,Float64,Int64,Int64,Int64,Float64,Int64,Bool,Bool,Bool,Bool
1,619,42,2,0.0,1,1,1,1.01349e5,1,1,0,0,1
2,608,41,1,83807.9,1,0,1,1.12543e5,0,0,1,0,1
3,502,42,8,1.59661e5,3,1,0,1.13932e5,1,1,0,0,1
4,699,39,1,0.0,2,0,0,93826.6,0,1,0,0,1
5,850,43,2,1.25511e5,1,1,1,79084.1,0,0,1,0,1


In [9]:
# Train Test Split
using Lathe.preprocess: TrainTestSplit
train, test = TrainTestSplit(df,.75)

(7424×13 DataFrame
│ Row  │ CreditScore │ Age   │ Tenure │ Balance   │ NumOfProducts │ HasCrCard │ IsActiveMember │ EstimatedSalary │ Exited │ France │ Spain │ Germany │ Female │
│      │ Int64       │ Int64 │ Int64  │ Float64   │ Int64         │ Int64     │ Int64          │ Float64         │ Int64  │ Bool   │ Bool  │ Bool    │ Bool   │
├──────┼─────────────┼───────┼────────┼───────────┼───────────────┼───────────┼────────────────┼─────────────────┼────────┼────────┼───────┼─────────┼────────┤
│ 1    │ 608         │ 41    │ 1      │ 83807.9   │ 1             │ 0         │ 1              │ 1.12543e5       │ 0      │ 0      │ 1     │ 0       │ 1      │
│ 2    │ 502         │ 42    │ 8      │ 1.59661e5 │ 3             │ 1         │ 0              │ 1.13932e5       │ 1      │ 1      │ 0     │ 0       │ 1      │
│ 3    │ 699         │ 39    │ 1      │ 0.0       │ 2             │ 0         │ 0              │ 93826.6         │ 0      │ 1      │ 0     │ 0       │ 1      │
│ 4    │ 645         

In [10]:
fm = @formula(Exited ~ CreditScore + Age + Tenure + Balance + NumOfProducts + HasCrCard + IsActiveMember + EstimatedSalary + Female + France + Spain)
logit = glm(fm, train, Binomial(), ProbitLink())

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Array{Float64,1},Binomial{Float64},ProbitLink},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

Exited ~ 1 + CreditScore + Age + Tenure + Balance + NumOfProducts + HasCrCard + IsActiveMember + EstimatedSalary + Female + France + Spain

Coefficients:
────────────────────────────────────────────────────────────────────────────────────────
                        Coef.   Std. Error       z  Pr(>|z|)     Lower 95%     Upper 95%
────────────────────────────────────────────────────────────────────────────────────────
(Intercept)      -1.72721      0.166916     -10.35    <1e-24  -2.05436      -1.40006
CreditScore      -0.000484914  0.000184825   -2.62    0.0087  -0.000847164  -0.000122664
Age               0.0411077    0.00168775    24.36    <1e-99   0.0377998     0.0444156
Tenure           -0.00721246   0.00613412    -1.18    0.2397  -0.0192351     0.0048102
Balance           1.

In [11]:
prediction = predict(logit, test)

2576-element Array{Union{Missing, Float64},1}:
 0.13142783216052387
 0.15092365707480238
 0.3256393926879725
 0.027622541251092977
 0.08155989636273626
 0.01905266956877608
 0.03553180910775468
 0.4143731999111164
 0.058218052549496094
 0.039503627183306286
 0.26223345408709153
 0.1125423489993457
 0.29927804692569376
 ⋮
 0.018666912599848047
 0.19421935198847104
 0.16619662823507425
 0.24246386714055207
 0.1853069812349375
 0.14871407307381662
 0.4915232162008844
 0.09352131327627766
 0.1320005852948256
 0.05023326997603834
 0.07968205163742567
 0.34627457293015285